In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
conll = datasets.load_dataset("conll2003")
conll.keys()

Found cached dataset conll2003 (C:/Users/kiril/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

dict_keys(['train', 'validation', 'test'])

In [3]:
conll["test"][1]

{'id': '1',
 'tokens': ['Nadim', 'Ladki'],
 'pos_tags': [22, 22],
 'chunk_tags': [11, 12],
 'ner_tags': [1, 2]}

In [4]:
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [5]:
from sbe_vallib import NerSampler

In [6]:
train = {'X': [' '.join(i['tokens']) for i in conll['train']],
         'y_true': [i['ner_tags'] for i in conll['train']],
         'y_pred': [i['ner_tags'] for i in conll['train']]}

oos = {'X': [' '.join(i['tokens']) for i in conll['test']],
         'y_true': [i['ner_tags'] for i in conll['test']],
         'y_pred': [i['ner_tags'] for i in conll['test']]}

In [7]:
train['X'][:3], train['y_true'][:3]

(['EU rejects German call to boycott British lamb .',
  'Peter Blackburn',
  'BRUSSELS 1996-08-22'],
 [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0]])

In [8]:
sampler = NerSampler(train = train,
                     oos = oos)

In [9]:
sampler.set_seed(1, bootstrap=True)
sampler.oos['X'][:10]

['RE : $ 25,000,000',
 '- Five barges , 30-day open , mid-Mississippi ( McGregor and south ) bid at 160 percent , offered at 170 percent , no comparisons .',
 'Blanc collected his seventh career World Cup win with a two jump combined score of 238.36 points , easily beating Foucras , the overall World Cup aerials champion , who was a distant second with 223.60 .',
 'DALLAS 6 11 .353 9 1/2',
 'C. Cairns b Saqlain 10',
 'NY JETS 1 12 0 221 368',
 'Halftime 1-1 .',
 'WESTERN CONFERENCE',
 'Group D',
 'Rain earlier delayed the start of play by 30 minutes .']

In [10]:
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

In [11]:
pipe = pipeline(task='ner', model=model, tokenizer=tokenizer)

In [12]:
pipe(sampler.oos['X'][:10])

[[],
 [{'entity': 'B-LOC',
   'score': 0.8496239,
   'index': 11,
   'word': 'mid',
   'start': 30,
   'end': 33},
  {'entity': 'B-LOC',
   'score': 0.99894613,
   'index': 13,
   'word': 'Mississippi',
   'start': 34,
   'end': 45},
  {'entity': 'B-LOC',
   'score': 0.98926777,
   'index': 15,
   'word': 'McGregor',
   'start': 48,
   'end': 56}],
 [{'entity': 'B-PER',
   'score': 0.9988147,
   'index': 1,
   'word': 'Blanc',
   'start': 0,
   'end': 5},
  {'entity': 'B-MISC',
   'score': 0.99777263,
   'index': 6,
   'word': 'World',
   'start': 35,
   'end': 40},
  {'entity': 'I-MISC',
   'score': 0.9987795,
   'index': 7,
   'word': 'Cup',
   'start': 41,
   'end': 44},
  {'entity': 'B-PER',
   'score': 0.9993482,
   'index': 23,
   'word': 'F',
   'start': 114,
   'end': 115},
  {'entity': 'I-PER',
   'score': 0.94976074,
   'index': 24,
   'word': '##ou',
   'start': 115,
   'end': 117},
  {'entity': 'I-PER',
   'score': 0.97224516,
   'index': 25,
   'word': '##c',
   'start': 1

In [13]:
from sbe_vallib.validation.nlp.general_tests.data_quality.test_train_test_independence import train_test_independence_test

In [15]:
train_test_independence_test(sampler)

	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
7499 7499
(7499, 85)


100%|██████████| 200/200 [00:08<00:00, 24.76it/s]


{'semaphore': 'grey',
 'result_dict': {'result': {'Фактический gini': 0.18245832161197367,
   'Квантиль': 1,
   'Результат теста': 'Grey'}},
 'result_dataframes': [   Фактический gini  Квантиль Результат теста
  0          0.182458         1            Grey],
 'result_plots': []}